In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
tf.__version__

'2.0.0'

In [3]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

In [4]:
x_train.shape,x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [6]:
x_train.shape,x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [19]:
train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(500)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(500)

In [21]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1=Conv2D(32,3,activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [17]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

In [12]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [23]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions=model(images)
        loss=loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels,predictions)

In [24]:
@tf.function
def test_step(images,labels):
    predictions=model(images)
    t_loss=loss_object(labels,predictions)
    
    test_loss(t_loss)
    test_accuracy(labels,predictions)

In [25]:
type(x_train[0][0][0][0])

numpy.float64

In [26]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images,labels in train_ds:
        train_step(images,labels)
    
    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)
    
    template = 'Epoch {:.2f}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}'
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100,test_loss.result(),test_accuracy.result()*100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1.00, Loss: 0.30, Accuracy: 91.49, Test Loss: 0.10, Test Accuracy: 96.96
Epoch 2.00, Loss: 0.08, Accuracy: 97.68, Test Loss: 0.06, Test Accuracy: 98.00
Epoch 3.00, Loss: 0.05, Accuracy: 98.57, Test Loss: 0.06, Test Accuracy: 98.01
Epoch 4.00, Loss: 0.04, Accuracy: 98.99, Test Loss: 0.05, Test Accuracy: 98.28
Epoch 5.00, Loss: 0.02, Accuracy: 99.29, Test Loss: 0.05, Test Accuracy: 98.48


# fix : WARNING
tensorflow:Layer my_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

In [27]:
tf.cast(x_train,tf.float32),tf.cast(x_test,tf.float32)
x_train.shape,x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [28]:
train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(500)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(500)

In [29]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions=model(images)
        loss=loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels,predictions)

In [30]:
@tf.function
def test_step(images,labels):
    predictions=model(images)
    t_loss=loss_object(labels,predictions)
    
    test_loss(t_loss)
    test_accuracy(labels,predictions)

In [31]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images,labels in train_ds:
        train_step(images,labels)
    
    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)
    
    template = 'Epoch {:.2f}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}'
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100,test_loss.result(),test_accuracy.result()*100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1.00, Loss: 0.02, Accuracy: 99.56, Test Loss: 0.06, Test Accuracy: 98.22
Epoch 2.00, Loss: 0.01, Accuracy: 99.71, Test Loss: 0.05, Test Accuracy: 98.41
Epoch 3.00, Loss: 0.01, Accuracy: 99.79, Test Loss: 0.05, Test Accuracy: 98.42
Epoch 4.00, Loss: 0.01, Accuracy: 99.87, Test Loss: 0.05, Test Accuracy: 98.48
Epoch 5.00, Loss: 0.00, Accuracy: 99.93, Test Loss: 0.05, Test Accuracy: 98.61


# 改

In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D,BatchNormalization,ELU,Dropout
from tensorflow.keras import Model

In [33]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1=Conv2D(32,3)
        self.bn1=BatchNormalization()
        self.dp=Dropout(0.5)
        self.flatten = Flatten()
        self.d1 = Dense(128)
        self.bn2=BatchNormalization()
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.keras.layers.ReLU()(x)
        x = self.dp(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.bn2(x)
        x = tf.keras.layers.ReLU()(x)
        x = self.dp(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [34]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

In [35]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [36]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions=model(images)
        loss=loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels,predictions)

In [37]:
@tf.function
def test_step(images,labels):
    predictions=model(images)
    t_loss=loss_object(labels,predictions)
    
    test_loss(t_loss)
    test_accuracy(labels,predictions)

In [38]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images,labels in train_ds:
        train_step(images,labels)
    
    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)
    
    template = 'Epoch {:.2f}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}'
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100,test_loss.result(),test_accuracy.result()*100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1.00, Loss: 0.31, Accuracy: 90.97, Test Loss: 0.13, Test Accuracy: 96.43
Epoch 2.00, Loss: 0.09, Accuracy: 97.26, Test Loss: 0.07, Test Accuracy: 97.75
Epoch 3.00, Loss: 0.05, Accuracy: 98.42, Test Loss: 0.06, Test Accuracy: 98.05
Epoch 4.00, Loss: 0.04, Accuracy: 98.98, Test Loss: 0.05, Test Accuracy: 98.05
Epoch 5.00, Loss: 0.02, Accuracy: 99.35, Test Loss: 0.05, Test Accuracy: 98.39
